# Plotting co-expression pattern of ASCL1, PROX1, and FOXA2 in malignant cells

## Setup Directories 

In [ ]:
import scanpy as sc
from common_utils import setup_dirs, find_arial_font, add_gene_binary_status
import os
import numpy as np

find_arial_font()

outDir = OUTDIR_CO_EXPRESSION
figuresDir, dataDir, tablesDir = setup_dirs(outDir)

sc.settings.figdir = figuresDir
sc.set_figure_params(dpi_save=300, vector_friendly=True)

## Add a binary gene expression status and plot the noramlized expression for the putative malignant cells

In [ ]:
# Constants
cmap = 'plasma'
genes = ["ASCL1", "FOXA2", "PROX1"]
adata_path = ADATA_CO_EXPRESSION_PATH

In [ ]:
adata = sc.read_h5ad(adata_path)
adata = adata[:, genes].copy()

# Add the binary status
for gene in genes:
    adata = add_gene_binary_status(adata, gene, threshold=0, use_counts=True)

# Create the 8 groups based on the expression of these genes
adata.obs["Expression Status"] = (
    adata.obs["ASCL1_is_expressed"].astype(str)
    + adata.obs["FOXA2_is_expressed"].astype(str)
    + adata.obs["PROX1_is_expressed"].astype(str)
)

# Map to readable names:  TrueTrueFalse -> ASCL1+FOXA2+PROX1-
for gene in genes:
    the_map = {True: f"{gene}+", False: f"{gene}-"}
    adata.obs[f"{gene}_status"] = adata.obs[f"{gene}_is_expressed"].map(the_map)

adata.obs["Expression Status"] = (
    adata.obs["ASCL1_status"].astype(str)
    + adata.obs["FOXA2_status"].astype(str)
    + adata.obs["PROX1_status"].astype(str)
)

order = adata.obs.sort_values("ASCL1_EXPR", ascending=False).index.tolist()
adata = adata[order].copy()

vmax = adata.X.max()
sc.pl.heatmap(
    adata,
    genes,
    groupby="Expression Status",
    show=False,
    swap_axes=True,
    figsize=(10, 5),
    save=f"_co_expression_19_status_{cmap}_vmax_{vmax}.pdf",
    use_raw=False,
    vmax=vmax,
    dendrogram=False,
    cmap=cmap,
)